# Airflow Integration Example - Bridge Failure Pipeline

In [1]:
from airflow import DAG
from airflow.operators.bash import BashOperator
from datetime import datetime, timedelta

## Define a Minimal Airflow DAG (for .py import, not execution in notebook)

In [2]:
default_args = {
    'owner': 'airflow',
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

with DAG(
    'bridge_failure_pipeline',
    default_args=default_args,
    description='A pipeline for bridge failure prediction',
    schedule_interval='@daily',
    start_date=datetime(2024, 1, 1),
    catchup=False,
) as dag:
    fetch_task = BashOperator(
        task_id='fetch_data',
        bash_command='python scripts/fetch_data.py'
    )
    feature_task = BashOperator(
        task_id='feature_engineering',
        bash_command='python scripts/feature_engineering.py'
    )
    train_task = BashOperator(
        task_id='train_model',
        bash_command='python scripts/train_model.py'
    )
    eval_task = BashOperator(
        task_id='evaluate_model',
        bash_command='python scripts/evaluate_model.py'
    )

    fetch_task >> feature_task >> train_task >> eval_task